In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from proj1_helpers import *
from Hugues import *

Data exploration and cleaning
===

Load data

In [3]:
from proj1_helpers import *

labels, raw_data, indices = load_csv_data('train.csv', sub_sample=False)
labels_te, raw_data_te, indices_te = load_csv_data('test.csv', sub_sample=False)

Divide data

In [4]:
labels0, data0, labels1, data1, labels2, data2 = divide_data(labels, raw_data)
labels0_te, data0_te, labels1_te, data1_te, labels2_te, data2_te = divide_data(labels_te, raw_data_te)

print('Train:', '0 jet', data0.shape, ', 1 jet', data1.shape, ', 2 jet', data2.shape)
print('Test:', '0 jet', data0_te.shape, ', 1 jet', data1_te.shape, ', 2 jet', data2_te.shape)

Train: 0 jet (99913, 30) , 1 jet (77544, 30) , 2 jet (72543, 30)
Test: 0 jet (227458, 30) , 1 jet (175338, 30) , 2 jet (165442, 30)


Remove useless columns

In [5]:
# enlever les features avec variance 0
clean_data0, clean_data0_te = remove_novar_features(data0, data0_te)
clean_data1, clean_data1_te = remove_novar_features(data1, data1_te)
clean_data2, clean_data2_te = remove_novar_features(data2, data2_te)

Columns with variance = 0 : (array([ 4,  5,  6, 12, 22, 23, 24, 25, 26, 27, 28, 29]),)
New data shape : (99913, 18)
Columns with variance = 0 : (array([ 4,  5,  6, 12, 22, 26, 27, 28]),)
New data shape : (77544, 22)
Columns with variance = 0 : (array([], dtype=int64),)
New data shape : (72543, 30)


Standardize data

In [6]:
std_data0, mean0, std0 = standardize_train(clean_data0) 
std_data0_te = standardize_test(clean_data0_te, mean0, std0)

std_data1, mean1, std1 = standardize_train(clean_data1)
std_data1_te = standardize_test(clean_data1_te, mean1, std1)

std_data2, mean2, std2 = standardize_train(clean_data2)
std_data2_te = standardize_test(clean_data2_te, mean2, std2)

Estimation of feature 0

In [7]:
estimated_data0, weights_train0 = column_estimation_train(std_data0)
estimated_data0_te = column_estimation_test(std_data0_te, weights_train0)

estimated_data1, weights_train1 = column_estimation_train(std_data1)
estimated_data1_te = column_estimation_test(std_data1_te, weights_train1)

estimated_data2, weights_train2 = column_estimation_train(std_data2)
estimated_data2_te = column_estimation_test(std_data2_te, weights_train2)

26123 NaN lines found
7562 NaN lines found
4429 NaN lines found


Find best parameters
===

In [8]:
### Parameters
degrees = [0, 1, 2]
lambdas = np.logspace(-1, -10, 10)
k_fold = 10
seed = 23

best_degree0, best_lambda0 = find_best_parameters(labels0, estimated_data0, k_fold, lambdas, degrees, seed)
best_degree1, best_lambda1 = find_best_parameters(labels1, estimated_data1, k_fold, lambdas, degrees, seed)
best_degree2, best_lambda2 = find_best_parameters(labels2, estimated_data2, k_fold, lambdas, degrees, seed)

Best degree: 2 Best lambda: 1e-05 Best score: 0.8229406465819238 Best loss: 0.08794915423881493
Best degree: 2 Best lambda: 0.01 Best score: 0.7588341501160691 Best loss: 0.1232138251225174
Best degree: 2 Best lambda: 1e-08 Best score: 0.7775020678246485 Best loss: 0.1165701681830714


Prediction
===

In [9]:
# Train model to get weights and predict on test
y_pred0 = make_predictions(estimated_data0, labels0, estimated_data0_te, best_lambda0, best_degree0)
y_pred1 = make_predictions(estimated_data1, labels1, estimated_data1_te, best_lambda1, best_degree1)
y_pred2 = make_predictions(estimated_data2, labels2, estimated_data2_te, best_lambda2, best_degree0)

In [11]:

labels_te[np.where(raw_data_te[:,22] == 0)[0]] = y_pred0
labels_te[np.where(raw_data_te[:,22] == 1)[0]] = y_pred1
labels_te[np.where(raw_data_te[:,22] > 1)[0]] = y_pred2

create_csv_submission(indices_te, labels_te, 'test_jet.csv')

(227458,)
